### Base Model: Llama3.2-Instruct
### Original Parameters: 3,237,063,680 = 3B 
### Trainable Parameters: 24,313,856 (0.75% trained) = 24M
### Technique Used : QLORA Fine Tuning

In [ ]:
! pip install -q unsloth
! pip install -q --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
! pip install -q datasets accelerate bitsandbytes trl


### Load model with Unsloth (4-bit QLoRA)

In [ ]:
! pip install unsloth_zoo

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Maximum context length (tokens) the model will accept
dtype = None           # auto-detect (torch.float16 or torch.bfloat16)
load_in_4bit = True    # QLoRA

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3.2-3b-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


### Apply LoRA adapters (VERY IMPORTANT)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)


### Load your dataset (JSONL)

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files="/content/training_data_fixed.jsonl",
    split="train"
)

print(dataset[0])


Convert to training text (Llama-3 chat format)

In [ ]:
def format_chat(example):
    return {
        "text": tokenizer.apply_chat_template(
            example["messages"],
            tokenize=False,
            add_generation_prompt=False
        )
    }

dataset = dataset.map(format_chat)


### Finding efficient Tokens

In [ ]:
def count_tokens(dataset, tokenizer, field="text"):
    return sum(len(tokenizer(example[field]).input_ids)
               for example in dataset)


In [ ]:
print(count_tokens(dataset, tokenizer))

### Fine-tune using TRL SFTTrainer

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = True,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 100,              # 🔹 increase later
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
        report_to = "none",
    ),
)


### Start training

In [ ]:
trainer.train()


### Save LoRA adapter

In [ ]:
model.save_pretrained("llama3-qlora-adapter")
tokenizer.save_pretrained("llama3-qlora-adapter")


### Inference test

In [ ]:
FastLanguageModel.for_inference(model)

prompt = "How many entities are there in this project?"

inputs = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt}],
    tokenize=True,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 400,
    temperature = 0.7,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
FastLanguageModel.for_inference(model)

prompt = "What is the role of User class?"

inputs = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt}],
    tokenize=True,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 400,
    temperature = 0.7,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
FastLanguageModel.for_inference(model)

prompt = "Explain how spring security implement in this project in detail?"

inputs = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt}],
    tokenize=True,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 2000,
    temperature = 0.7,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [ ]:
FastLanguageModel.for_inference(model)

prompt = "Explain the workflow of this project?"

inputs = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt}],
    tokenize=True,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids = inputs,
    max_new_tokens = 2000,
    temperature = 0.7,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
